In [ ]:
!apt install libomp-dev
!pip install faiss-gpu

In [2]:
import faiss
import pandas as pd
import numpy as np

### 🪷 faiss metric으로 output token과 emoji의 유사성 계산

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

vectors = []
df = pd.read_csv('/content/drive/MyDrive/BOAZ/emoji_vector_unicode.csv')


In [ ]:
import torch

vector_set = df["vector"]

# change vector type from string to list
vector_set = vector_set.apply(lambda x: x[1:-1].split(', '))
vector_set = vector_set.apply(lambda x: [float(i) for i in x])


# change vector type from list to numpy array
vector_set = vector_set.apply(lambda x: np.array(x))

# change vector type from numpy array to torch tensor
# vector_set = torch.tensor(vector_set)

In [ ]:
vector_array = vector_set.astype(np.float32)

---

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("augustinLib/text-emoji-encoder")
model = AutoModelForSequenceClassification.from_pretrained("augustinLib/text-emoji-encoder")

In [ ]:
import torch

In [ ]:
input_sequence = "mouse"
tokenized_input = tokenizer(input_sequence, return_tensors="pt")
logits = model(input_ids = tokenized_input.input_ids,
               attention_mask = tokenized_input.attention_mask).logits

query_vector = logits.detach().numpy()

In [ ]:
dimension = 300
faiss_index = faiss.IndexFlatIP(dimension)
print(faiss_index.ntotal)

In [ ]:
faiss_index.add(vector_array)

In [ ]:
Distance, Index = faiss_index.search(query_vector, 3)

In [ ]:
print(f'Index:{Index}')
print(f'Distance:{Distance}')

In [ ]:
df.loc[Index[0], :]